### Import the libraries

In [1]:
# using Python 3.12.8

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
from scipy.stats import loguniform

import matplotlib.pyplot as plt

### Import the dataset
> Labelled Faces in the Wild Dataset

    people.csv:
> Contains randomly generated splits for 10-fold cross validation specifically for individual faces. 

> There are 10 total sets, each with a different amount of people:
> Set 1: 601. Set 2: 555. Set 3: 552. Set 4: 560. Set 5: 567. Set 6: 527. Set 7: 597. Set 8: 601. Set 9: 580. Set 10: 609.

In [ ]:
# Load the LFW dataset with a minimum of 70 images per person
# and resize the images to 40% of their original size

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes for plotting later on
n_samples, h, w = lfw_people.images.shape

X = lfw_people.data
n_features = X.shape[1]
y = lfw_people.target # Target values are the person's ID labels
target_names = lfw_people.target_names # Person's name

print("Number of samples: %d" % n_samples) # Number of samples = rows in X
print("Number of features: %d" % n_features) # Number of features = columns in X
print("Number of classes: %d" % len(target_names)) # Number of classes = number of unique values in y

### Functions

In [ ]:
# Apply PCA for dimensionality reduction
def apply_pca(X_train, X_test):
    n_components = 150
    print(
    "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
    )
    pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
    eigenfaces = pca.components_.reshape((n_components, h, w))

    print("Projecting the input data on the eigenfaces orthonormal basis")
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)

    return X_train_pca, X_test_pca, eigenfaces

In [ ]:
# Find the best SVM Model with GridSearchCV
def findTheBestSVMModel(X_train, y_train):
    # Define the parameter grid for GridSearchCV
    param_grid_svm = {
        'C': [0.1, 1],                   
        'gamma': [0.001, 0.01, 0.1],                
        'kernel': ['linear', 'rbf'],
        'class_weight': ['balanced', None]                    
    }

    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=10)

    svmModel_grid = GridSearchCV(
        estimator=SVC(probability=True, max_iter=5000),
        param_grid=param_grid_svm, 
        verbose=2,
        cv=cv, 
        n_jobs=4) # n_jobs=cores used, -1 = all cores
    
    svmModel_grid.fit(X_train, y_train)
    best_model = svmModel_grid.best_estimator_
    return best_model


In [ ]:
# Predict and evaluate the models
def predict_and_evaluate(model, X_test, y_test, target_names):
    y_pred = model.predict(X_test) # Predict the labels
    unique_labels = np.unique(y_test) # Get the unique labels
    print(classification_report(y_test, y_pred, labels=unique_labels, target_names=target_names, zero_division=0))
    print(confusion_matrix(y_test, y_pred, labels=unique_labels))
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')
    return y_pred

### Split the dataset
    train_test_split()

> Split **the dataset** into training and testing sets (20% for testing and 80% for training); 
> Choose random_state as a fixed seed value to ensure that the random processes (spliting the data) produce the same results every run; 
> Stratify the split to ensure that the same proportion of classes are present in both sets


In [ ]:
# Split the dataset into training and testing sets (20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Number of training samples: %d" % X_train.shape[0])
print("Number of testing samples: %d" % X_test.shape[0])

## Supervised Learning Method - Support Vector Machine (SVM)

### Scalling the data
    StandardScaler()

> Standardize the features by removing the mean and scaling to unit variance before splitting the data


In [ ]:
# Standardize the features by removing the mean and scaling to unit variance before splitting the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Number of training samples: %d" % X_train.shape[0])
print("Number of testing samples: %d" % X_test.shape[0])

### Applying PCA for dimensionality reduction

In [ ]:
# Apply PCA for dimensionality reduction
X_train_pca, X_test_pca, eigenfaces = apply_pca(X_train, X_test)

### Train and evaluate the SVM Model for original and PCA-reduced data

In [ ]:
# Train and evaluate SVM on original data
print("SVM on Original Data")
svm = findTheBestSVMModel(X_train, y_train)
y_pred_original = predict_and_evaluate(svm, X_test, y_test, target_names)

# Train and evaluate SVM on PCA data
print("SVM on PCA Data")
svm_pca = findTheBestSVMModel(X_train_pca, y_train)
y_pred_pca = predict_and_evaluate(svm_pca, X_test_pca, y_test, target_names)

SVM on Original Data
Fitting 10 folds for each of 24 candidates, totalling 240 fits
                   precision    recall  f1-score   support

     Ariel Sharon       0.67      0.75      0.71        16
     Colin Powell       0.86      0.94      0.90        47
  Donald Rumsfeld       0.72      0.75      0.73        24
    George W Bush       0.93      0.92      0.92       106
Gerhard Schroeder       0.72      0.82      0.77        22
      Hugo Chavez       0.83      0.71      0.77        14
       Tony Blair       0.91      0.72      0.81        29

         accuracy                           0.85       258
        macro avg       0.81      0.80      0.80       258
     weighted avg       0.86      0.85      0.85       258

[[12  0  1  1  1  1  0]
 [ 3 44  0  0  0  0  0]
 [ 1  1 18  1  1  0  2]
 [ 1  3  4 97  1  0  0]
 [ 0  0  1  2 18  1  0]
 [ 1  1  0  1  1 10  0]
 [ 0  2  1  2  3  0 21]]
Accuracy: 0.85
SVM on PCA Data
Fitting 10 folds for each of 24 candidates, totalling 240 fits
 

## Semi-supervised Learning Method - SelfTrainingClassifier with an SVM base estimator

### Splitting and Scalling for the Semi-Supervised SVM (SelfTrainingClassifier)
1. First test_train_split: 

> Split **the training set** into labeled (30%) and unlabeled (70%) sets 

2. Combine the training data

> Combine labeled and unlabeled **training data** for scalling and PCA.
> This step is necessary because semi-supervised learning requires access to both labeled and unlabeled data

3. StandardScaler():

> Standardize the features by removing the mean and scaling to unit variance before splitting the data



In [ ]:
# Split the training set into labeled (30%) and unlabeled (70%) sets
X_train_labeled, X_train_unlabeled, y_train_labeled, y_train_unlabeled = train_test_split(
    X_train, y_train, test_size=0.7, random_state=42)

# Combine labeled and unlabeled training data
X_train_combined = np.vstack((X_train_labeled, X_train_unlabeled))
y_train_combined = np.concatenate((y_train_labeled, [-1] * len(y_train_unlabeled))) # -1 for unlabeled samples

# Scale the combined training data
scaler = StandardScaler()
X_train_combined_scaled = scaler.fit_transform(X_train_combined)
X_test_scaled = scaler.transform(X_test)

print("Number of combined training samples: %d" % X_train_combined_scaled.shape[0])
print("Number of testing samples: %d" % X_test_scaled.shape[0])

### Applying PCA for dimensionality reduction

In [ ]:
# Apply PCA for dimensionality reduction
X_train_combined_pca, X_test_pca, eigenfaces = apply_pca(X_train_combined_scaled, X_test_scaled)

### Training the semi-supervised model using SelfTrainingClassifier with an SVM base estimator

1. Original Data

In [ ]:
# Train the semi-supervised model using SelfTrainingClassifier with an SVM base estimator
# On the original combined training data
base_estimator = findTheBestSVMModel(X_train_labeled, y_train_labeled)

# Print base_estimator to confirm the best parameters selected by findTheBestSVMModel
print(base_estimator)

# Train the SelfTrainingClassifier model on the original combined training data
self_training_model_original = SelfTrainingClassifier(
    estimator=base_estimator, 
    criterion='threshold', 
    threshold=0.95, 
    # k_best=10, 
    max_iter=20, 
    verbose=True
)

self_training_model_original_fit = self_training_model_original.fit(X_train_combined_scaled, y_train_combined)

# Passing the model to the predict_and_evaluate function for predicting the labels and evaluating the model
print("Self-Training Model on Original Data")
y_pred_self_training_original_final = predict_and_evaluate(
    self_training_model_original_fit, 
    X_test_scaled, 
    y_test, 
    target_names)



Fitting 10 folds for each of 24 candidates, totalling 240 fits
SVC(C=0.1, class_weight='balanced', gamma=0.001, kernel='linear', max_iter=5000,
    probability=True)
End of iteration 1, added 49 new labels.
End of iteration 2, added 3 new labels.
End of iteration 3, added 1 new labels.
End of iteration 4, added 6 new labels.
Self-Training Model on Original Data
                   precision    recall  f1-score   support

     Ariel Sharon       0.60      0.56      0.58        16
     Colin Powell       0.86      0.89      0.88        47
  Donald Rumsfeld       0.79      0.62      0.70        24
    George W Bush       0.82      0.92      0.87       106
Gerhard Schroeder       0.59      0.59      0.59        22
      Hugo Chavez       1.00      0.50      0.67        14
       Tony Blair       0.68      0.66      0.67        29

         accuracy                           0.78       258
        macro avg       0.76      0.68      0.71       258
     weighted avg       0.79      0.78      

2. PCA-reduced data

In [ ]:
# Train the semi-supervised model
# On the PCA-reduced combined training data
self_training_model_pca = SelfTrainingClassifier(base_estimator, criterion='threshold', threshold=0.95, max_iter=20, verbose=True)
self_training_model_pca.fit(X_train_combined_pca, y_train_combined)
y_pred_self_training_pca = self_training_model_pca.predict(X_test_pca)

In [ ]:
print("Self-Training Model on PCA Data")
y_pred_self_training_pca_final = predict_and_evaluate(self_training_model_pca, X_test_pca, y_test, target_names)

Self-Training Model on PCA Data
                   precision    recall  f1-score   support

     Ariel Sharon       0.67      0.38      0.48        16
     Colin Powell       0.77      0.79      0.78        47
  Donald Rumsfeld       0.78      0.58      0.67        24
    George W Bush       0.73      0.89      0.80       106
Gerhard Schroeder       0.54      0.59      0.57        22
      Hugo Chavez       1.00      0.50      0.67        14
       Tony Blair       0.78      0.62      0.69        29

         accuracy                           0.73       258
        macro avg       0.75      0.62      0.66       258
     weighted avg       0.74      0.73      0.72       258

[[ 6  1  1  8  0  0  0]
 [ 1 37  0  6  3  0  0]
 [ 0  3 14  6  0  0  1]
 [ 2  5  0 94  3  0  2]
 [ 0  1  2  4 13  0  2]
 [ 0  0  0  5  2  7  0]
 [ 0  1  1  6  3  0 18]]
Accuracy: 0.73


## Results

In [ ]:
# Plot some of the test results
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return f'predicted: {pred_name}\ntrue:      {true_name}'

prediction_titles_original = [title(y_pred_original, y_test, target_names, i) for i in range(y_pred_original.shape[0])]
prediction_titles_pca = [title(y_pred_pca, y_test, target_names, i) for i in range(y_pred_pca.shape[0])]

plot_gallery(X_test, prediction_titles_original, h, w)
plt.show()

plot_gallery(X_test, prediction_titles_pca, h, w)
plt.show()